In [2]:
import pandas as pd

hate = pd.read_csv("all_examples_hate.csv", sep = "\t")
nonhate = pd.read_csv("all_examples_nonhate.csv", sep = "\t")

In [59]:
hate['label'] = 1
nonhate['label'] = 0

combined = pd.concat([hate, nonhate])

combined_file_path = "all_examples.csv"
combined.to_csv(combined_file_path, sep="\t", index=False)

In [60]:
guideline_counts= combined['guideline'].value_counts()
guideline_counts

guideline
hatecrime victim                                       367
violent speech                                         284
better worse than                                      255
curse profane                                          254
contempt self admission intolerance                    242
spread virus                                           235
attacking intellectual capability                      231
harmful stereotype                                     228
economic exclusion                                     221
less than adequate                                     218
criminal                                               212
disgust repulsion                                      208
curse sexual                                           208
explicit exclusion                                     205
deviating norm                                         199
attacking derogatory term                              199
subhumanity                                   

In [61]:
guideline_counts= combined['dataset'].value_counts()
guideline_counts

dataset
dynahate          5128
gpt               1384
hatecheck          442
manual             270
asonam             152
toxic span         100
hate offensive      91
twitter             33
FRENK               19
ethos               11
Name: count, dtype: int64

# Replace sheetname

In [53]:
import pandas as pd

hate = pd.read_csv("all_examples_hate_raw.csv", sep = "\t")
nonhate = pd.read_csv("/data/jzheng36/HateModerate/postprocess/all_examples_nonhate_raw.csv", sep = "\t")

nonhate['guideline'] = nonhate['guideline'].replace('contempt self admission intoler', 'contempt self admission intolerance')
nonhate['guideline'] = nonhate['guideline'].replace('attack concept associated prote', 'attack concept associated protected characteristics')
nonhate['guideline'] = nonhate['guideline'].replace('attacking intellectual capabili', 'attacking intellectual capability')

hate['dataset'] = hate['dataset'].replace('comments', 'toxic span')
hate['dataset'] = hate['dataset'].replace('data', 'FRENK')
hate['dataset'] = hate['dataset'].replace('davidson', 'hate offensive')

output_path = "/data/jzheng36/HateModerate/postprocess/all_examples_nonhate_raw.csv"
nonhate.to_csv(output_path, sep="\t", index=False)

output_path = "/data/jzheng36/HateModerate/postprocess/all_examples_hate_raw.csv"
hate.to_csv(output_path, sep="\t", index=False)

# add gpt to test

In [20]:
import pandas as pd

test = pd.read_csv("/data/jzheng36/HateModerate/fine_tune/datasets/testing/hatemoderate_test.csv", sep = "\t")
gpt = pd.read_csv("all_examples_hate.csv.csv", sep = "\t")

gpt = gpt[gpt['dataset'] == 'gpt']

columns_to_keep = ['example_id', 'text', 'guideline', 'labels']
gpt = gpt[columns_to_keep]

test_updated = pd.concat([test, gpt], ignore_index=True)

test_updated.to_csv("/data/jzheng36/HateModerate/fine_tune/datasets/testing/hatemoderate_test_updated.csv", sep="\t", index=False)

In [4]:
def process_df(df):
    # Group by 'guideline' and count the number of rows without 'gpt' in 'dataset'
    counts = df[df['dataset'] != 'gpt'].groupby('guideline').size()

    # Filter rows
    for guideline, count in counts.items():
        if count >= 50:
            # Remove rows with 'gpt'
            df = df[~((df['guideline'] == guideline) & (df['dataset'] == 'gpt'))]
        else:
            # Reduce the number of rows including 'gpt' to 50
            total_to_keep = 50
            non_gpt_count = df[(df['guideline'] == guideline) & (df['dataset'] != 'gpt')].shape[0]
            gpt_to_keep = total_to_keep - non_gpt_count
            # If there are already less than 50 rows, keep them as is
            if gpt_to_keep > 0:
                # Randomly select gpt rows to keep
                gpt_rows = df[(df['guideline'] == guideline) & (df['dataset'] == 'gpt')]
                gpt_rows_kept = gpt_rows.sample(min(gpt_to_keep, len(gpt_rows)), random_state=1)
                # Keep only the selected gpt rows and all non-gpt rows
                df = df[(df['guideline'] != guideline) | (df['dataset'] != 'gpt') | (df.index.isin(gpt_rows_kept.index))]
    
    return df

processed_hate = process_df(hate)
processed_hate_file = "all_examples_hate.csv"

processed_hate.to_csv(processed_hate_file, sep="\t", index=False)

In [31]:
import pandas as pd

# Load the test dataset
test = pd.read_csv("/data/jzheng36/HateModerate/fine_tune/datasets/testing/hatemoderate_test.csv", sep="\t")

# Filter for hate (labels = 1) and non-hate (labels = 0)
hate_test = test[test['labels'] == 1]
nonhate_test = test[test['labels'] == 0]

# Save the filtered datasets to new CSV files
hate_test.to_csv("/data/jzheng36/hatemoderate/hatemoderate/step2/hatemoderate_test_hate.csv", sep="\t", index=False)
nonhate_test.to_csv("/data/jzheng36/hatemoderate/hatemoderate/step2/hatemoderate_test_nonhate.csv", sep="\t", index=False)


In [30]:
ids_to_remove.count()

1